Settings

In [1]:
start_date_str = "2023-01-01" # Start date of data
end_date_str = "2025-12-31" # End date of data (inclusive)

# Filenames
demanda_ejecutado_filename = "demanda_ejecutado.csv"
prog_dia_filename = "prog_dia.csv"
reprog_dia_filename = "reprog_dia.csv"
demanda_real_filename = "demanda_real.csv"

Download process

In [21]:
import datetime
import os
import unicodedata
from io import BytesIO

import pandas as pd
from downloader.coes_downloader import (
    get_demanda_ejecutado,
    get_medidores_generacion,
    get_urls_prog_dia,
    get_urls_reprog_dia,
    process_demanda_ejecutado,
    session,
)
from tqdm import tqdm


In [5]:
data_dir = "data/"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [3]:
# Set start date of every month in the range
starts = pd.date_range(
    start_date_str, end_date_str, freq="MS",
).to_series().reset_index(drop=True)

# Set end date of every month in the range
ends = pd.date_range(
    start_date_str, end_date_str, freq="ME",
).to_series().reset_index(drop=True)

# Concatenate series
date_ranges = pd.concat([starts, ends], axis=1)
date_ranges.columns = ["start_date", "end_date"]

# Set threshold date of programa and reprograma
start_date_threshold = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")  # noqa: DTZ007
start_date_threshold = start_date_threshold - datetime.timedelta(days=1)


In [ ]:
# Set filters to be applied after download
start_date_filter = pd.to_datetime(start_date_str, format="%Y-%m-%d") + pd.Timedelta(minutes=15)
end_date_filter = pd.to_datetime(end_date_str, format="%Y-%m-%d") + pd.Timedelta(days=1)
start_date_filter, end_date_filter

(Timestamp('2023-01-01 00:15:00'), Timestamp('2026-01-01 00:00:00'))

In [ ]:
demnanda_ejecutado_dfs = []
demnanda_real_dfs = []

# Iterate date ranges and download ejecutado and demanda real
for _, (start_date, end_date) in tqdm(date_ranges.iterrows()):
    start_date_obj = start_date.to_pydatetime().date()
    end_date_obj = end_date.to_pydatetime().date()

    # Request demanda ejecutado
    demanda_ejecutado_response = get_demanda_ejecutado(
        start=start_date_obj,
        end=end_date_obj,
    )

    # Process request to dataframe
    demanda_ejecutado = process_demanda_ejecutado(demanda_ejecutado_response)

    # Request medidores de generacion. generacion = demanda
    demanda_real = get_medidores_generacion(
        start=start_date_obj,
        end=end_date_obj,
    )

    # Append to dataframes list
    demnanda_ejecutado_dfs.append(demanda_ejecutado)
    demnanda_real_dfs.append(demanda_real)

# Concatenate ejecutado dataframes and rename
df_ejecutado = pd.concat(demnanda_ejecutado_dfs, axis=0)
df_ejecutado = df_ejecutado.rename(
    columns={
        "Fecha": "timestamp",
        "ValorEjecutado": "demanda_ejecutado",
        "ValorProgramacionDiaria": "programacion_diaria",
        "ValorProgramacionSemanal": "reprogramacion_diaria",
    },
)
df_ejecutado = df_ejecutado.set_index("timestamp")
df_ejecutado.index = df_ejecutado.index.tz_localize(None) # timezone naive
df_ejecutado = df_ejecutado.sort_index().loc[start_date_filter:end_date_filter]

# Concatenate dataframes and sum generation to get demanada real
df_demnanda_real = pd.concat(demnanda_real_dfs, axis=0)
df_demnanda_real = df_demnanda_real.rename(columns={"fechahora": "timestamp"})
df_demnanda_real = df_demnanda_real.set_index("timestamp")
df_demnanda_real = df_demnanda_real.sort_index().loc[start_date_filter:end_date_filter]

# Sum all of the generation to get demanda real
df_demnanda_real = df_demnanda_real.sum(axis=1).to_frame("demanda_real")

36it [02:53,  4.83s/it]


In [119]:
# Save ejecutado and demanda_real csvs in data directory
df_ejecutado.to_csv(os.path.join(data_dir, demanda_ejecutado_filename))
df_demnanda_real.to_csv(os.path.join(data_dir, demanda_real_filename))

In [16]:
# Crawl through programa diario endpoint and get download urls
prog_dia_endpoints = get_urls_prog_dia(
    start_date_threshold=start_date_threshold.date(),
)

# Crawl through reprograma diario endpoint and get download urls
reprog_dia_endpoints = get_urls_reprog_dia(
    start_date_threshold=start_date_threshold.date(),
)

In [46]:
# Filter excel file urls
prog_dia_excel_files = list(
    filter(
        lambda x: x[1].lower().strip().lstrip().startswith("anexo1_despacho"),
        prog_dia_endpoints,
    ),
)

reprog_dia_excel_files = list(
    filter(
        lambda x: x[2].lower().strip().lstrip().startswith("reprog_"),
        reprog_dia_endpoints,
    ),
)

In [ ]:
prog_dia_dfs = []
for date, _, download_url in tqdm(prog_dia_excel_files):
    response = session.get(
        "https://www.coes.org.pe/Portal/browser/download",
        params={"url": download_url},
    )

    df_prog = pd.read_excel(BytesIO(response.content), skiprows=5)
    df_prog_cols = df_prog.columns.to_list()
    df_prog_cols = [i.lower().strip().lstrip().lower() for i in df_prog_cols]

    df_prog_cols = [unicodedata.normalize("NFD", i)
                    .encode("ascii", "ignore")
                    .decode("utf-8").replace(" ", "_")
                    for i in df_prog_cols]
    df_prog.columns = df_prog_cols

    if "dia_hora" not in df_prog_cols:
        timestamp_col_name = "hora"
    else:
        timestamp_col_name = "dia_hora"

    df_prog = df_prog[[timestamp_col_name, "oferta"]].dropna(how="any")

    df_prog = df_prog.iloc[:48]

    df_prog[timestamp_col_name] = pd.date_range(
        date, date + datetime.timedelta(days=1), freq="30min", inclusive="left",
    ) + pd.Timedelta(minutes=30)

    df_prog = df_prog.rename(
        columns={timestamp_col_name: "timestamp", "oferta": "prog_dia"},
    )
    prog_dia_dfs.append(df_prog)

df_prog_dia = pd.concat(prog_dia_dfs, axis=0)
df_prog_dia = df_prog_dia.set_index("timestamp").sort_index()
df_prog_dia = df_prog_dia.loc[start_date_filter:end_date_filter]

# Save prog_dia in data directory
df_prog_dia.to_csv(os.path.join(data_dir, prog_dia_filename))

100%|██████████| 1113/1113 [03:02<00:00,  6.09it/s]


In [ ]:
reprog_dia_dfs = []

for date, version, _, download_url in tqdm(reprog_dia_excel_files):
    response = session.get(
        "https://www.coes.org.pe/Portal/browser/download",
        params={"url": download_url},
    )

    df_prog = pd.read_excel(BytesIO(response.content), skiprows=5)
    df_prog_cols = df_prog.columns.to_list()
    df_prog_cols = [i.lower().strip().lstrip().lower() for i in df_prog_cols]

    df_prog_cols = [unicodedata.normalize("NFD", i)
                    .encode("ascii", "ignore")
                    .decode("utf-8").replace(" ", "_")
                    for i in df_prog_cols]
    df_prog.columns = df_prog_cols

    if "dia_hora" not in df_prog_cols:
        timestamp_col_name = "hora"
    else:
        timestamp_col_name = "dia_hora"

    df_prog = df_prog[[timestamp_col_name, "oferta"]].dropna(how="any")
    if len(df_prog) > 0:
        start_timestamp = str(df_prog[timestamp_col_name].iloc[0])

        time_range = pd.date_range(
            date.strftime("%Y-%m-%d") + " " + start_timestamp,
            (date + datetime.timedelta(days=1)).strftime("%Y-%m-%d"),
            freq="30min",
        )
        df_prog = df_prog.iloc[:len(time_range)]
        df_prog[timestamp_col_name] = time_range

        df_prog = df_prog.rename(
            columns={timestamp_col_name: "timestamp", "oferta": "reprog_dia"},
        )

        df_prog["version"] = version
        reprog_dia_dfs.append(df_prog)
    else:
        print(f"{date} {version} empty file")

df_reprog_dia = pd.concat(reprog_dia_dfs, axis=0)
df_reprog_dia = df_reprog_dia.sort_values(["timestamp", "version"])
df_reprog_dia = df_reprog_dia.pivot_table(
    index="timestamp", columns="version", values="reprog_dia",
)
df_reprog_dia.columns.name = None
df_reprog_dia = df_reprog_dia.loc[start_date_filter:end_date_filter]
df_reprog_dia = df_reprog_dia.reindex(
    pd.date_range(
        df_reprog_dia.index[0], df_reprog_dia.index[-1], freq="30min",
    ),
)
df_reprog_dia.index.name = "timestamp"
# Save reprog_dia in data directory
df_reprog_dia.to_csv(os.path.join(data_dir, reprog_dia_filename))

 46%|████▋     | 1971/4249 [07:31<07:17,  5.20it/s]  

2024-09-25 D empty file


100%|██████████| 4249/4249 [16:58<00:00,  4.17it/s]  
